The goal of this notebook is to develop an end2end text classifier using different embedding like word2vec, fasttext, and doc2vec.

The first pipe is to clean the text data (More cleaning for future data.) 

In [38]:
# Data cleaning
# ===========================================================================
import re
def clean_text(text):
    # replace  . and a space with only a space, then amke all words lower case.
    text = text.replace(". "," ").replace(",","").lower()
    # get rid of the . at the end of each line. 
    cleaned_text = re.sub("\.$","",text)
    
    return cleaned_text
 


class text_clean:
    """
    A class to help with cleaning text data. 
    """
    def fit(self, X, y):
        return self
    def transform(self, X):
        assert isinstance(X,pd.Series), "The input data should be pandas Series."
        X = X.apply(clean_text)
        
        return X


# Word embedding training 
# ===========================================================================
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.externals import joblib



def _find_part_pii(text, model, sep = " "):
    tokenized_text = text.split(sep)
    
    part_pii = model.wv.doesnt_match(tokenized_text)
    
    return part_pii    



def _extracted_pii2matrix(pii_list, model):
    # set the matrix dimensions
    column_num = model.trainables.layer1_size
    row_num = len(pii_list)
    # initialized the matrix
    pii2vec_mat = np.zeros((row_num, column_num))
    # iterate through the pii_list and assign the vectors to matrix.
    for index, ith_pii in enumerate(tqdm(pii_list)):
        pii2vec_mat[index,:] = model.wv[ith_pii]
    
    return pii2vec_mat



class word_embedding:
    """
    A class to convert words/docs to vectors by applying any model supported by gensim.  
    
    This class will allow continued training on the pre-trained model by assigning
    the model to the pre_trained option in class initialization.  
    
    After training the model, it will dump the word2vec model to the path assigned to 
    dump_file option.  
    
    
    """
    def __init__(self, algo_name = "word2vec", size = 100, min_count = 1, window = 5, workers =1,\
                 epochs = 5, pre_train = None, dump_file = False,\
                 re_train_new_sentences = True):
        
        
        assert algo_name in ["word2vec", 'fasttext', 'doc2vec'], \
        "please enter a model name in ['word2vec', 'fasttext', 'doc2vec']"
        
        self.algo_name = algo_name
        self.epochs = epochs 
        self.pre_train = pre_train
        self.dump_file = dump_file 
        self.re_train_new_sentences = re_train_new_sentences
        
        # model options
        self.size = size
        self.min_count = min_count
        self.window = window
        self.workers = workers
        
        
    def _algo_init(self):
        if self.algo_name == "word2vec":
            model = Word2Vec(size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
        elif self.algo_name == "fasttext":
            model = FastText(size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
        elif self.algo_name == "doc2vec":
            model = Doc2Vec(vector_size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
            
        self.model = model
        return self

    def _embedding_training(self, sentences, update = False):
        """
        if update = True, it will update the vocabulary and the model can continue to train.
        If update = False, the model will rebuild a new vocabulary from scratch using the input data.
        """
        updated_model_with_vocab = self.model

        updated_model_with_vocab.build_vocab(sentences, update = update)
        
        updated_model_with_vocab.train(sentences, total_examples = len(sentences), epochs = self.epochs)
        
        # update the model with the trained one. 
        self.model = updated_model_with_vocab
        
    def _pd_to_gensim_format(self, text):
        
        # special handling for doc2vec model. 
        if self.algo_name == "doc2vec":
            documents = [TaggedDocument(sentence.split(" "), [index])\
                          for index, sentence in enumerate(text)] 
            print("Using index for the tags")    
        else:
            documents = [sentence.split(" ") for sentence in text]
            
            
        return documents
            
        
    def fit(self, X):
        """
        The fit method will get use the pre_trained model if the model is assigned to the pre_train option.
        
        If the pre_train is None, then the model will be trained. 
        """
        gensim_X = self._pd_to_gensim_format(text = X)
        
        if self.pre_train is not None:
            self.model = self.pre_train
            return self
        else:
            # initialize the model, split the sentence into tokens and train it. 
            self._algo_init()
            self._embedding_training(sentences = gensim_X)
            
        return self
        
    
    def transform(self, X):
        """
        If re_train_new_sentences is True, which is the default setting, 
        the model will be re-trained on the new sentences. 
        This will create word embedding for words not in the original vocabulary.
        This will increase the model inference time since it invovles model training. 
        
        For using word2vec to predict PII data, it is recommended to update the model with new sentences. 
        For fastttext, it is not necessary since it will infer from the character n-grams. The fasttext training
        is much longer than word2vec. 
        """
        gensim_X = self._pd_to_gensim_format(text = X)
        # update the embedding with new sentences or train the model. 
        if self.re_train_new_sentences:
            self._embedding_training(sentences = gensim_X, update = True)
            print("transforming while training {} model with new data.".format(self.algo_name))
            
            
        # extract the PII 
        extracted_pii_list = [_find_part_pii(text = text, model = self.model)\
                    for text in tqdm(X) ]
        
        # convert the extracted pii text into vectors.
        piivec_matrix = _extracted_pii2matrix(pii_list = extracted_pii_list,\
                                          model = self.model)
        return piivec_matrix 
                                          

In [39]:
import pandas as pd
algo_test_data = pd.read_csv("../data/train_text_with_pii_2019_01_05_02_48_24_796403.csv")

## Word2vec testing

In [40]:
testing_embedding = word_embedding(algo_name = 'word2vec')
testing_embedding.fit(algo_test_data['Text']);
test_pii_matrix = testing_embedding.transform(algo_test_data["Text"])
test_pii_matrix

transforming while training word2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 59669.30it/s]


array([[-0.00552159, -0.00203225, -0.00489316, ..., -0.00241043,
         0.00955387, -0.01075913],
       [-0.00626223, -0.00024075, -0.00917153, ..., -0.00296824,
         0.00430816, -0.00740499],
       [-0.00933292,  0.00148898, -0.00867963, ..., -0.00629046,
         0.00426485, -0.00276585],
       ...,
       [-0.02212575,  0.00074998, -0.02750545, ..., -0.02030576,
         0.02142496, -0.02265347],
       [-0.02438817,  0.00431983, -0.01989118, ..., -0.01755816,
         0.01750636, -0.0174949 ],
       [-0.01403672,  0.00744611, -0.02118406, ..., -0.00960288,
         0.0180083 , -0.01796213]])

In [41]:
pre_trained = Word2Vec.load("./word2vec/word2vec_cleaned_300_.bin")
testing_pre_trained = word_embedding(algo_name = "word2vec", pre_train = pre_trained)
testing_pre_trained.fit(algo_test_data["Text"])
testing_pre_trained.transform(algo_test_data["Text"])


transforming while training word2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 41656.13it/s]


array([[ 0.07526994, -0.18667389, -0.96105301, ...,  0.15214525,
         0.02959202, -0.41313222],
       [ 0.42987594, -0.05497638, -0.96129489, ...,  0.46197411,
         0.22145101, -0.54831982],
       [ 0.04325041, -0.0285845 , -0.00989333, ..., -0.05394333,
         0.20068049,  0.08983152],
       ...,
       [-0.04774014, -0.05916891,  0.03238036, ..., -0.08683169,
         0.08028586,  0.00404379],
       [ 0.29659271, -0.58653063,  0.01698974, ...,  0.43566033,
        -0.06582996,  1.55989099],
       [ 0.15159334, -0.12143414, -0.10422158, ...,  0.03164278,
         0.05929553,  0.09101988]])

## Fasttext testing

In [42]:
testing_embedding = word_embedding(algo_name = 'fasttext')

testing_embedding.fit(algo_test_data["Text"]);
test_pii_matrix = testing_embedding.transform(algo_test_data["Text"])
test_pii_matrix

transforming while training fasttext model with new data.


100%|██████████| 800/800 [00:00<00:00, 89131.47it/s]


array([[-0.00797034,  0.00529751, -0.02028651, ..., -0.05967641,
        -0.00291255,  0.05439368],
       [ 0.0006045 ,  0.00365394, -0.01218145, ..., -0.03478827,
        -0.00018567,  0.030108  ],
       [-0.00948382,  0.00533016, -0.03240121, ..., -0.08218741,
        -0.00651771,  0.07036647],
       ...,
       [-0.02091836,  0.01299771, -0.05696366, ..., -0.15254602,
        -0.0053024 ,  0.13926317],
       [-0.02786561,  0.01796342, -0.06887486, ..., -0.17057717,
        -0.00203732,  0.15210037],
       [-0.02402436,  0.0121314 , -0.06329311, ..., -0.15296748,
        -0.00263037,  0.13743119]])

## doc2vec testing

In [43]:
testing_embedding_doc2vec = word_embedding(algo_name = 'doc2vec')
testing_embedding_doc2vec.fit(X = algo_test_data['Text'])

test__doc2vec_pii_matrix = testing_embedding_doc2vec.transform(algo_test_data['Text'])
test__doc2vec_pii_matrix

Using index for the tags
Using index for the tags
transforming while training doc2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 93113.64it/s]


array([[-0.00335109, -0.00195854, -0.00548842, ..., -0.00846553,
        -0.00111197, -0.00612434],
       [-0.0051255 , -0.00264326, -0.00670908, ..., -0.00396664,
         0.00131375, -0.00429768],
       [-0.0112185 , -0.00281456, -0.00480118, ..., -0.00295908,
         0.00372413, -0.00662092],
       ...,
       [-0.01494441, -0.00622322, -0.01718825, ..., -0.02094651,
         0.01046414, -0.01136488],
       [-0.02091397, -0.00221723, -0.01378535, ..., -0.02076681,
         0.0110327 , -0.00925473],
       [-0.01569927,  0.00378848, -0.01961735, ..., -0.01383845,
         0.00869041, -0.0135015 ]])

In [10]:
## add transfer_learning option.

In [11]:
## add dum_file option